<a href="https://colab.research.google.com/github/JacopoMangiavacchi/FastLabeling/blob/master/mnist_odd_even_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai -U

Requirement already up-to-date: fastai in /opt/anaconda3/lib/python3.7/site-packages (2.0.0)


In [2]:
from fastai.vision.all import *

matplotlib.rc('image', cmap='Greys')

In [3]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
path.ls()

(#2) [Path('training'),Path('testing')]

In [4]:
(path/'training').ls()

(#10) [Path('training/9'),Path('training/0'),Path('training/7'),Path('training/6'),Path('training/1'),Path('training/8'),Path('training/4'),Path('training/3'),Path('training/2'),Path('training/5')]

In [5]:
from shutil import copy2, rmtree

def copy_files(path, source_folder, destination_folder, sub_folders_list = [], random_select = False, max_files_to_copy = 0):
  def copy_folder_path(source_path, destination_path):
    source_files = source_path.ls()
    list_files = list(range(len(source_files)))
    if random_select:
      random.shuffle(list_files)
    count = 0
    for i in list_files:
      copy2(source_files[i], destination_path)
      count += 1
      if max_files_to_copy > 0 and count >= max_files_to_copy:
        break
  
  (path/destination_folder).mkdir(exist_ok=True)
  if len(sub_folders_list) > 0:
    for i in sub_folders_list:
      folder = source_folder + '/' + str(i)
      copy_folder_path(path/folder, path/destination_folder)
  else:
    copy_folder_path(path/source_folder, path/destination_folder)


In [6]:
rmtree(path/'oe_training', ignore_errors=true)
rmtree(path/'oe_testing', ignore_errors=true)

(path/'oe_training').mkdir(exist_ok=True)
(path/'oe_testing').mkdir(exist_ok=True)
path.ls()

(#4) [Path('oe_testing'),Path('training'),Path('oe_training'),Path('testing')]

In [7]:
copy_files(path, 'training', 'oe_training/odd', [0, 2, 4, 6, 8])
copy_files(path, 'training', 'oe_training/even', [1, 3, 5, 7, 9])
copy_files(path, 'testing', 'oe_testing/odd', [0, 2, 4, 6, 8])
copy_files(path, 'testing', 'oe_testing/even', [1, 3, 5, 7, 9])


In [8]:
(path/'oe_training/odd').ls(), (path/'oe_training/even').ls(), (path/'oe_testing/odd').ls(), (path/'oe_testing/even').ls()

((#29492) [Path('oe_training/odd/20037.png'),Path('oe_training/odd/3975.png'),Path('oe_training/odd/59292.png'),Path('oe_training/odd/17843.png'),Path('oe_training/odd/16585.png'),Path('oe_training/odd/27980.png'),Path('oe_training/odd/22620.png'),Path('oe_training/odd/45863.png'),Path('oe_training/odd/8820.png'),Path('oe_training/odd/34042.png')...],
 (#30508) [Path('oe_training/even/49081.png'),Path('oe_training/even/38678.png'),Path('oe_training/even/30224.png'),Path('oe_training/even/27758.png'),Path('oe_training/even/51816.png'),Path('oe_training/even/29775.png'),Path('oe_training/even/26446.png'),Path('oe_training/even/39566.png'),Path('oe_training/even/21329.png'),Path('oe_training/even/36655.png')...],
 (#4926) [Path('oe_testing/odd/8820.png'),Path('oe_testing/odd/9280.png'),Path('oe_testing/odd/3961.png'),Path('oe_testing/odd/6831.png'),Path('oe_testing/odd/3949.png'),Path('oe_testing/odd/4798.png'),Path('oe_testing/odd/374.png'),Path('oe_testing/odd/7291.png'),Path('oe_testin

In [9]:
size_traning = 600
size_testing = 100
size_training_name = str(size_traning) + '_training'
size_testing_name = str(size_testing) + '_testing'

rmtree(path/size_training_name, ignore_errors=true)
rmtree(path/size_testing_name, ignore_errors=true)

(path/size_training_name).mkdir(exist_ok=True)
(path/size_testing_name).mkdir(exist_ok=True)
path.ls()

(#6) [Path('oe_testing'),Path('100_testing'),Path('training'),Path('oe_training'),Path('600_training'),Path('testing')]

In [10]:
for i in range(10):
  copy_files(path, 'training/' + str(i), size_training_name + '/' + str(i), random_select=True, max_files_to_copy=size_traning)
  copy_files(path, 'testing/' + str(i), size_testing_name + '/' + str(i), random_select=True, max_files_to_copy=size_testing)

In [11]:
dls = ImageDataLoaders.from_folder(path, train='oe_training', valid='oe_testing')

dls.train_ds, dls.valid_ds

((#60000) [(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0))...],
 (#10000) [(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size=28x28, TensorCategory(0)),(PILImage mode=RGB size

In [12]:
learn = cnn_learner(dls, resnet18, metrics=accuracy)
learn.fine_tune(3)


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/jacopo/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


epoch,train_loss,valid_loss,accuracy,time
0,0.324690,0.246632,0.902500,04:18


epoch,train_loss,valid_loss,accuracy,time
0,0.056844,0.033190,0.988000,07:44
1,0.022443,0.016404,0.994000,07:42
2,0.010216,0.014793,0.994700,07:40


In [13]:
dls2 = ImageDataLoaders.from_folder(path, train=size_training_name, valid=size_testing_name)

dls2.train_ds, dls2.valid_ds

((#6000) [(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9))...],
 (#1000) [(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=28x28, TensorCategory(9)),(PILImage mode=RGB size=2

In [14]:
learn2 = cnn_learner(dls2, resnet18, metrics=accuracy, custom_head=learn.model[0])
learn2.fine_tune(10)


epoch,train_loss,valid_loss,accuracy,time
0,1.964314,1.105582,0.698000,00:07


epoch,train_loss,valid_loss,accuracy,time
0,0.909383,0.577909,0.831000,00:08
1,0.563732,0.398360,0.880000,00:08
2,0.349674,0.256505,0.924000,00:08
3,0.219905,0.271853,0.924000,00:08
4,0.166739,0.215932,0.950000,00:08
5,0.143255,0.189715,0.944000,00:08
6,0.107148,0.119198,0.972000,00:08
7,0.086068,0.155990,0.961000,00:08
8,0.060578,0.129421,0.966000,00:08
9,0.058056,0.109178,0.974000,00:08


In [15]:
test_path = (path/'testing'/'5').ls()[10]
test_image = Image.open(test_path)
test_image

In [16]:
learn2.predict(test_path)

('5',
 tensor(5),
 tensor([3.6981e-09, 1.8678e-08, 7.0219e-09, 3.7110e-07, 1.7352e-08, 1.0000e+00,
         3.2858e-08, 1.9617e-09, 3.7087e-08, 4.8641e-08]))

In [17]:
learn3 = cnn_learner(dls2, resnet18, metrics=accuracy)
learn3.fine_tune(10)

epoch,train_loss,valid_loss,accuracy,time
0,1.949910,1.072911,0.693000,00:07


epoch,train_loss,valid_loss,accuracy,time
0,0.942511,0.573510,0.830000,00:08
1,0.547251,0.385032,0.880000,00:08
2,0.336105,0.261480,0.918000,00:08
3,0.236321,0.203848,0.940000,00:08
4,0.171206,0.218231,0.939000,00:08
5,0.121971,0.232856,0.940000,00:08
6,0.100468,0.147275,0.956000,00:08
7,0.093202,0.145344,0.967000,00:08
8,0.056697,0.164508,0.957000,00:08
9,0.046969,0.141605,0.964000,00:08


In [22]:
learn.model[1][8]

Linear(in_features=512, out_features=2, bias=False)

In [23]:
learn.model[1][8] = torch.nn.modules.linear.Linear(in_features=512, out_features=10, bias=False)

In [24]:
# learn4 = Learner(dls2, learn.model, metrics=[accuracy], loss_func=CrossEntropyLossFlat(), splitter=default_split)
learn4 = Learner(dls2, learn.model, metrics=accuracy)
learn4.fine_tune(10)


epoch,train_loss,valid_loss,accuracy,time
0,0.498180,0.287498,0.947000,00:45


epoch,train_loss,valid_loss,accuracy,time
0,0.129795,0.056863,0.984000,00:46
1,0.130975,0.606362,0.860000,00:46
2,0.170571,0.122231,0.963000,00:45
3,0.111177,0.115795,0.973000,00:45
4,0.082663,0.121869,0.965000,00:45
5,0.067688,0.093055,0.977000,00:45
6,0.042897,0.063502,0.983000,00:45
7,0.029227,0.048269,0.986000,00:46
8,0.016103,0.042474,0.988000,00:46
9,0.008659,0.040452,0.989000,00:47
